<a href="https://colab.research.google.com/github/TYH71/AIML_CA1/blob/main/Hyperaparameter_Optimization_(LGBM).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
! pip install -q --upgrade imbalanced-learn
import imblearn
from tqdm.notebook import tqdm

# Machine Learning Modules
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

     |████████████████████████████████| 199 kB 13.4 MB/s 


In [3]:
# Matplotlib styling
plt.style.use('seaborn-dark')
plt.rcParams['figure.figsize'] = 16, 9
plt.rcParams['figure.autolayout'] = True

## Hardware Specifications

|![Colab Logo](https://colab.research.google.com/img/colab_favicon_256px.png)|
|:--:|
|Google Colaboratory Logo [[Source](https://blog.tensorflow.org/2018/05/colab-easy-way-to-learn-and-use-tensorflow.html)]|

Having the right resources is important to any data science project, which provides an increase in performance and quality. In our case, every team member has purchased [Colab Pro](https://colab.research.google.com/signup) for better GPUs and memory, in order to avoid any bottlenecks in computational performances.



In [4]:
# Retrieve CPU and GPU information
! lscpu
! nvidia-smi

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              4
On-line CPU(s) list: 0-3
Thread(s) per core:  2
Core(s) per socket:  2
Socket(s):           1
NUMA node(s):        1
Vendor ID:           AuthenticAMD
CPU family:          23
Model:               49
Model name:          AMD EPYC 7B12
Stepping:            0
CPU MHz:             2249.998
BogoMIPS:            4499.99
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            512K
L3 cache:            16384K
NUMA node0 CPU(s):   0-3
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ht syscall nx mmxext fxsr_opt pdpe1gb rdtscp lm constant_tsc rep_good nopl nonstop_tsc cpuid extd_apicid tsc_known_freq pni pclmulqdq ssse3 fma cx16 sse4_1 sse4_2 movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm cmp_legacy cr8_legacy abm sse4a misalignss

In [5]:
# Mounting Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Metadata

The telematics data collected during driving trips are useful to detect if the trip is dangerous and has high crash / accident probability.

Data Schema (Field & Description):

| bookingID | trip |
| ---:|:--- |
| accuracy | accuracy inferred by GPS in meters |
| bearing | GPS bearing in degree |
| acceleration_x | accelerometer reading at x axis (m/s2) |
| acceleration_y | accelerometer reading at y axis (m/s2) |
| acceleration_z | accelerometer reading at z axis (m/s2) (Acceleration determines the acceleration / vibration of the device in motion. Each of the axis can be thought of as a different sensor even though they reside on the same physical chip) |
| gyro_x | gyroscope reading in x axis (rad/s) |
| gyro_y | gyroscope reading in y axis (rad/s) |
| gyro_z | gyroscope reading in z axis (rad/s) (Gyroscope data determine the orientation of the device relative to earth's gravity) |
| second | time of the record by number of seconds |
| speed | speed measured by GPS in m/s |
| label | tags to indicate dangerous driving trip (0: Normal trip / 1: Dangerous trip) |

!! IMPORTANT NOTE: ACCURACY MEANS THE DISTANCE BETWEEN THE PHONE LOCATION AND WHERE IT IS ACTAULLY LOCATED !!

In [6]:
# Import dataset
df = pd.read_csv('/content/drive/MyDrive/PAI Documents/Processed Data/processed_telematic_data.csv')
display(df)

,BookingID,Second,Accuracy,Bearing,Acceleration_x,Acceleration_y,Acceleration_z,Gyro_x,Gyro_y,Gyro_z,Speed,Label
0,0,2.0,8.0,143.29830,-1.706207,-9.270792,-1.209448,-0.028965,-0.032652,0.015390,0.228454,0
1,0,3.0,8.0,143.29830,-1.416705,-9.548032,-1.860977,-0.022413,0.005049,-0.025753,0.228454,0
2,0,9.0,8.0,143.29830,-0.346924,-9.532629,-1.204663,0.014962,-0.050033,0.025118,0.228454,0
3,0,11.0,8.0,143.29830,-0.600986,-9.452029,-2.157507,0.004548,-0.011713,-0.004078,0.228454,0
4,0,12.0,8.0,143.29830,-0.597546,-9.863403,-1.672711,-0.000401,0.000315,-0.009830,0.228454,0
...,...,...,...,...,...,...,...,...,...,...,...,...
7018380,1709396983975,552.0,6.0,185.44942,-0.654221,-8.646927,-4.137964,0.017065,-0.127480,-0.079793,1.695058,1
7018381,1709396983975,553.0,6.0,194.25633,-0.260342,-8.792276,-4.570722,-0.018650,-0.115063,-0.046133,1.695058,1
7018382,1709396983975,554.0,7.0,198.52795,0.179144,-8.934634,-4.572067,-0.015760,-0.018210,-0.006609,1.035811,1
7018383,1709396983975,555.0,8.0,198.52795,-0.585284,-9.059647,-3.930856,-0.013872,0.009509,-0.008706,1.035811,1


In [7]:
# custom function for measure of dispersion
def standard_deviation():
    def standard_deviation_(x):
        return np.std(x, ddof=0)
    standard_deviation_.__name__ = 'std'
    return standard_deviation_

# aggregate by bookingid
df_agg = pd.DataFrame()
for col in tqdm(df.columns):
    if col != "BookingID" and col != "Label":
        # temp = df.groupby("BookingID")[col].agg(["mean", percentile(99), percentile(1), standard_deviation()])
        temp = df.groupby("BookingID")[col].agg(["mean", 'max', 'min', standard_deviation()])
        df_agg[col + "_mean"] = temp["mean"]
        df_agg[col + "_max"] = temp["max"]
        df_agg[col + "_min"] = temp["min"]
        df_agg[col+ "_std"] = temp['std']
    elif col == "Label":
        temp = df.groupby("BookingID")[col].agg(["max"])
        df_agg[col] = temp['max']

# display dataframe
df_agg.reset_index(inplace=True)
display(df_agg)

# display dataframe shape
print("Before Aggregation: {}".format(df.shape))
print("After Aggregation: {}".format(df_agg.shape))
print("Rows Pct Change: {:.2%}".format((df.shape[0] - df_agg.shape[0])/df.shape[0]))

  0%|          | 0/12 [00:00<?, ?it/s]

,BookingID,Second_mean,Second_max,Second_min,Second_std,Accuracy_mean,Accuracy_max,Accuracy_min,Accuracy_std,Bearing_mean,Bearing_max,Bearing_min,Bearing_std,Acceleration_x_mean,Acceleration_x_max,Acceleration_x_min,Acceleration_x_std,Acceleration_y_mean,Acceleration_y_max,Acceleration_y_min,Acceleration_y_std,Acceleration_z_mean,Acceleration_z_max,Acceleration_z_min,Acceleration_z_std,Gyro_x_mean,Gyro_x_max,Gyro_x_min,Gyro_x_std,Gyro_y_mean,Gyro_y_max,Gyro_y_min,Gyro_y_std,Gyro_z_mean,Gyro_z_max,Gyro_z_min,Gyro_z_std,Speed_mean,Speed_max,Speed_min,Speed_std,Label
0,0,928.337054,1589.0,2.0,520.366462,9.787946,16.000,4.000,3.484934,176.962977,359.97977,0.037464,130.147436,-0.706297,4.782614,-4.692294,0.923042,-9.606546,-6.119916,-12.190784,0.654424,-1.534789,2.318857,-6.251807,1.155939,0.002182,0.197898,-0.392537,0.065548,-0.003596,0.428893,-0.516878,0.093217,-0.001543,0.338149,-0.210122,0.060552,9.127778,22.882523,0.064309,7.051503,0
1,1,575.399485,1034.0,0.0,280.954077,3.703376,6.000,3.000,0.559644,127.372423,337.00000,25.000000,89.368574,-0.503200,3.387767,-5.352994,0.780670,9.533105,11.435962,6.623425,0.521352,-2.202569,1.009146,-5.145321,0.846832,-0.001151,0.126536,-0.100130,0.026410,-0.004788,0.470837,-0.600795,0.098411,0.002577,0.136467,-0.119799,0.032313,7.729992,21.882141,0.000000,6.925198,1
2,2,320.777778,822.0,10.0,346.005531,3.821759,8.000,3.000,1.102356,179.475309,330.00000,11.000000,116.019046,0.287461,1.956122,-1.565856,0.762748,9.735713,10.901134,8.248277,0.429384,0.069421,1.755003,-2.825244,0.953170,-0.001514,0.282162,-0.155218,0.053593,-0.013358,0.344953,-0.418629,0.109132,0.004042,0.112942,-0.081313,0.031953,2.851901,9.257438,0.000000,2.762353,1
3,4,541.207039,1092.0,1.0,320.002049,10.000000,10.000,10.000,0.000000,150.385762,351.40985,2.271227,69.543971,-0.380176,2.019635,-2.866458,0.572697,-9.371676,-7.135416,-12.916333,0.472537,-2.654539,0.296381,-5.029947,0.768237,-0.023892,0.148668,-0.230107,0.034797,0.023619,0.652628,-0.389871,0.117423,0.002339,0.505220,-0.209906,0.065074,6.048292,19.560000,0.000000,5.423302,1
4,6,540.685437,1092.0,1.0,319.443463,4.631772,12.000,3.000,1.360049,195.625243,346.00000,0.000000,113.862376,0.565690,5.008823,-2.013585,0.794527,9.504979,13.249917,0.594051,0.763701,2.314074,6.263423,-1.438959,0.934951,0.006719,0.362202,-0.200405,0.055024,-0.009303,0.434384,-0.621802,0.109557,0.001852,0.258855,-0.336128,0.058501,4.714045,16.394695,0.000000,5.329937,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19927,1709396983957,533.340090,1044.0,0.0,304.085661,3.549740,14.601,3.000,0.987798,174.596847,359.00000,0.000000,88.372367,0.103698,11.296188,-15.993755,2.493784,1.827002,39.845520,-34.944440,5.210744,9.778801,42.316414,-24.088799,5.886929,0.027901,2.139516,-0.866552,0.225325,-0.013217,0.935871,-1.425123,0.168826,-0.012467,1.884930,-2.603097,0.378629,2.483309,6.275580,0.000290,1.709695,1
19928,1709396983960,424.188623,809.0,2.0,236.289830,11.808383,16.000,4.000,3.848842,178.551648,350.57666,10.027151,71.121502,0.560114,3.709992,-3.379367,0.786324,-8.704715,-6.629535,-11.460449,0.657741,-4.471745,-1.930212,-7.562341,0.862148,-0.012863,0.290076,-0.641901,0.075406,0.006026,0.477245,-0.376187,0.091944,-0.004241,0.233631,-0.496440,0.050454,7.593701,24.059150,0.000000,7.021465,1
19929,1709396983966,584.032258,988.0,1.0,267.643560,9.379500,13.936,5.953,1.257486,197.983871,359.00000,0.000000,80.907884,-0.201059,2.232147,-2.704697,0.735621,9.062923,11.092163,7.785767,0.414098,3.336205,5.963333,0.272308,0.699459,0.001111,0.122772,-0.172302,0.024953,-0.005624,0.392319,-0.683594,0.104167,-0.001112,0.151810,-0.252991,0.043086,12.739501,25.640000,0.000000,7.481683,1
19930,1709396983971,523.682773,1076.0,1.0,304.385036,4.773118,16.000,3.000,1.734899,205.259454,357.00000,1.000000,105.138591,0.733790,4.637710,-3.550709,1.028630,8.901846,11.430268,5.542746,0.592276,3.772809,9.347249

Before Aggregation: (7018385, 12)
After Aggregation: (19932, 42)
Rows Pct Change: 99.72%


In [8]:
# Define Features and Labels
X = df_agg.set_index('BookingID').drop('Label', axis=1)
y = df_agg.set_index('BookingID').loc[:, 'Label']

print("X shape: {}\ny shape: {}".format(X.shape, y.shape))

X shape: (19932, 40)
y shape: (19932,)


In [9]:
# Split Training and Testing Ratio - 4:1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("X_train: {}\ty_train: {}\nX_test: {}\ty_test: {}".format(X_train.shape, y_train.shape, X_test.shape, y_test.shape))

X_train: (15945, 40)	y_train: (15945,)
X_test: (3987, 40)	y_test: (3987,)


# Setting up Ray Parallel Backend

> In Topic 4 - Big Data Resource Management, we've learnt to use Dask parallelism for machine learning training and prediction in a parallel manner. Applying this concept, we did further research on the best technique for backend distributed training. We have found that Ray parallel backend the best method for distributed training, of 2 times faster for SVM Grid Search and 8 times faster for Random Forest algorithm.

![Backend Comparison](https://miro.medium.com/max/1400/1*wN0Xe6mRHZyg7ubRV8HnYA.png)

In some way, think of `Ray` as an alternative to `Dask`.

In [10]:
# prepare ray parallel backend
import joblib
!pip install -q ray
from ray.util.joblib import register_ray
import ray
ray.init(ignore_reinit_error=True)
register_ray()

     |████████████████████████████████| 59.6 MB 86.8 MB/s 
     |████████████████████████████████| 173 kB 62.1 MB/s 


# LGBM Pipeline

In [12]:
# pipeline and preprocessing modules
from imblearn.pipeline import Pipeline
from sklearn.neighbors import LocalOutlierFactor
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler

## Local Outlier Factor

Previously, we have found that there are quite a number outliers in the raw dataset. We went through A/B testing to find the best method/algorithm. We have found that `LocalOutlierFactor` is the best algorithm for handling outliers.

In [14]:
# Identify outliers via LocalOutlierFactor
iso = LocalOutlierFactor(contamination=0.05)
X_out = iso.fit_predict(X_train)
# select all rows that are not outliers
mask = (X_out != -1)

# Store removed outliers in a new variable
X_train_out, y_train_out = X_train[mask], y_train[mask]

# summarize the shape of the updated training dataset
print("Shape of the training dataset before removing outliers", X_train.shape, y_train.shape)
print('Shape of the training dataset after removing outliers', X_train_out.shape, y_train_out.shape)

Shape of the training dataset before removing outliers (15945, 40) (15945,)
Shape of the training dataset after removing outliers (15147, 40) (15147,)


## SMOTE

Previously, we have found that the labels are heavily imbalanced in classes. We went through A/B testing to find the best method/algorithm. We have found that `SMOTE` is the best algorithm for handling imbalance classes.


In [15]:
# instantiate smote class and resample
smote = SMOTE(sampling_strategy='auto', random_state=42, k_neighbors=5, n_jobs=-1)
X_train_smote, y_train_smote = smote.fit_resample(X_train_out, y_train_out)

print("Before Resampling: {} {}".format(X_train_out.shape, y_train_out.shape))
print("After Resampling: {} {}".format(X_train_smote.shape, y_train_smote.shape))

Before Resampling: (15147, 40) (15147,)
After Resampling: (22938, 40) (22938,)


## Z-Score Standardization

> Since all of the features are numerical-continuous values, we can use Z-Score Standardization to standardize the data to a Gaussian distribution $N(\mu=0, \sigma=1)$. The intuition of standardization is to ensure all feature remains relatively on the same scale, so that the machine learning algorithm (espescially Linear and Support Vector Models) would not overfit over a particular feature.

$$ Z = \frac{X - \mu}{\sigma} $$

In [ ]:
# instantiate standard scaler class
scaler = StandardScaler()
X_train_scale = scaler.fit_transform(X_train_smote, y_train_smote)
X_train_scale = pd.DataFrame(X_train_scale, columns=X_train_smote.columns)

## Create Pipeline Function

In [16]:
# function to build a machine learning pipeline
def create_pipeline(model):
    # model: expects an estimator object, being able to .fit() and .predict()
    # returns a pipeline instance
    ml_pipeline = Pipeline(
        steps=[
            ("SMOTE", smote),
            ("Z-Score Standardization", scaler),
            (model.__repr__(), model)
        ]
    )
    return ml_pipeline


## LGBM Pipeline Development

In [17]:
# yellowbrick - machine learning visualisation library
! pip install -q yellowbrick
from yellowbrick.classifier import ROCAUC, PrecisionRecallCurve, ConfusionMatrix
from yellowbrick.model_selection import LearningCurve

# metrics
from sklearn.metrics import make_scorer, precision_score, accuracy_score, f1_score, recall_score
from sklearn.model_selection import cross_validate
precision = make_scorer(precision_score, pos_label=1)
recall = make_scorer(recall_score, pos_label=1)
accuracy = make_scorer(accuracy_score)
f1 = make_scorer(f1_score, pos_label=1)
metrics = {
    "Accuracy": accuracy,
    "F1": f1,
    "Precision": precision,
    "Recall": recall
}

In [18]:
# Function to create Classification Plot
def classification_plot(model):
    with plt.style.context("seaborn-darkgrid"):
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12, 12), tight_layout=True)
        rocauc = ROCAUC(
            model, fontsize=18, binary=True, encoder={0: "Normal Trip", 1: "Dangerous Trip"}, ax=ax1
        )
        rocauc.fit(X_train_out, y_train_out)
        rocauc.score(X_test, y_test)
        rocauc.finalize()

        prc = PrecisionRecallCurve(
            model, fontsize=18, binary=True, encoder={0: "Normal Trip", 1: "Dangerous Trip"}, ax=ax2
        )
        prc.fit(X_train_out, y_train_out)
        prc.score(X_test, y_test)
        prc.finalize()

        prc = ConfusionMatrix(
            model, encoder={0: "Normal Trip", 1: "Dangerous Trip"}, ax=ax3, percent=True, cmap='Blues'
        )
        prc.fit(X_train_out, y_train_out)
        prc.score(X_test, y_test)
        prc.finalize()

        visualizer = LearningCurve(
            model, cv=10, scoring='f1', n_jobs=-1, fontsize=18, ax=ax4
        )
        visualizer.fit(X_train_out, y_train_out)
        visualizer.finalize()

    plt.show()

In [ ]:
from xgboost import XGBClassifier
# function to build a machine learning pipeline
def create_pipeline(model):
    # model: expects an estimator object, being able to .fit() and .predict()
    # returns a pipeline instance
    ml_pipeline = Pipeline(
        steps=[
            ("SMOTE", smote),
            ("Z-Score Standardization", scaler),
            (model.__repr__(), model)
        ]
    )
    return ml_pipeline

# attempting to recreate ml baseline
xgboost_pipeline = create_pipeline(XGBClassifier(objective='binary:logistic', n_jobs=-1))
print(xgboost_pipeline)